In [5]:
from pre_processing import df_all, df_week_player,df_week_team, metrics_classes
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import numpy as np
import re
import os
import datetime
import altair as alt

In [6]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [7]:
df_all["Date"].min()

Timestamp('2021-07-01 00:00:00')

In [8]:
df_ori = pd.read_csv("./data/anonymous.csv")

In [9]:
df_ori.head()

,date,player,position,team_name,duration,total_distance_m,total_player_load,acc_2m_s_s_total_efforts,acc_3m_s_s_total_efforts,dec_2m_s_s_total_efforts,dec_3m_s_s_total_efforts,high_intensity_distance_m_v5_v6_m,sprint_distance_m_m,maximum_velocity_km_h,ima_cod_left,ima_cod_right
0,01/07/2021,Player1,Central Midfielder,Team1,102,5162.0,491.0,3,0,7,2,96.7,0.0,25.7,50,39
1,01/07/2021,Player2,Central Midfielder,Team1,190,4932.9,561.4,14,1,12,3,170.0,0.0,25.0,43,42
2,06/07/2021,Player3,Winger,Team2,66,5628.6,626.4,18,2,11,1,223.8,39.1,30.0,33,23
3,06/07/2021,Player4,Central Midfielder,Team3,55,6239.1,624.9,18,1,9,7,332.4,12.2,29.3,17,18
4,06/07/2021,Player5,Attacker,Team3,55,5440.4,507.1,22,3,14,8,299.6,6.1,29.2,19,40


In [10]:
df_ori_processed = df_ori[df_ori["duration"]>=15]
training_count = df_ori_processed["player"].value_counts()

In [11]:
len(df_ori) - len(df_ori_processed)

207

In [12]:
df_ori_processed_2 = df_ori_processed[df_ori_processed['player'].isin(training_count[training_count>7].index)]

In [13]:
len(df_ori_processed) - len(df_ori_processed_2)

187

In [14]:
len(df_ori),  len(df_ori_processed), len(df_ori_processed_2)

(24826, 24619, 24432)

In [15]:
df_all.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volume Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
0,2021-07-01,Player1,Central Midfielder,Team1,102.0,5162.0,491.0,3.0,0.0,7.0,2.0,96.7,0.0,7.138889,50.0,39.0,2021/22,Thursday,26,2021,2021-W26,4.81,50.61,0.97,0.22,True,Right,43.82%,56.18%,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
67160,2021-07-01,Player88,Centre Back,Team3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
159870,2021-07-01,Player172,Winger,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
139430,2021-07-01,Player104,Full Back,Team6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
87600,2021-07-01,Player115,Centre Back,Team5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Thursday,26,2021,2021-W26,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0


In [16]:
df_week_player.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volume Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,7.138889,50.0,39.0,4.81,50.61,0.97,0.22,True,Right,43.82%,56.18%,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,7.527778,15.8,19.0,9.38,99.17,1.28,-0.20,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,1,0,0,2,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,7.611111,24.8,27.6,9.05,84.34,1.53,-0.11,True,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,7.844444,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,0,2,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,7.622222,22.4,25.4,8.56,87.94,1.24,-0.13,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0,2021-07-26,2021/22


In [17]:
player1 = df_week_player[df_week_player["Player"]=="Player1"]
player1.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volume Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Date,Season
0,Player1,Central Midfielder,Team1,2021,26,2021-W26,102.0,5162.00,491.00,3.0,0.0,7.0,2.0,96.70,0.00,7.138889,50.0,39.0,4.81,50.61,0.97,0.22,True,Right,43.82%,56.18%,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,7.527778,15.8,19.0,9.38,99.17,1.28,-0.20,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,1,0,0,2,2021-07-05,2021/22
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,7.611111,24.8,27.6,9.05,84.34,1.53,-0.11,True,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,7.844444,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,0,2,2021-07-19,2021/22
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,7.622222,22.4,25.4,8.56,87.94,1.24,-0.13,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0,2021-07-26,2021/22


In [18]:
not_pass_metrics = {}
for key, value in metrics_classes.items():
    not_pass = [metric for metric in value if any(player1[f"is_{metric}_abnormal"]!="Moderate")]
    not_pass_metrics[key] = not_pass

In [19]:
not_pass_metrics

{'Volume': ['Total Distance(m)', 'Duration'],
 'Intensity': ['Acc-Dec-COD Per Minute'],
 'Agility': ['Acc 2m/s2 Total Effort',
  'Acc 3m/s2 Total Effort',
  'Dec 2m/s2 Total Effort',
  'Dec 3m/s2 Total Effort'],
 'IMA': ['IMA COD(left)', 'IMA COD(right)']}

In [20]:
plot_df = df_week_player[df_week_player["Player"] == "Player1"].dropna()

In [21]:
ewma_acwr = alt.Chart(plot_df).mark_line(point=alt.OverlayMarkDef(color="#FF7676"), color="#FA8072").encode(
alt.X("Date"),
alt.Y(f'Total Distance(m) EWMA ACWR'),
alt.Tooltip(["Date:T", "Total Distance(m) EWMA ACWR", "Total Distance(m):Q"])
)

# Create the dashed horizontal line at y = 1.5
upper_ewma_acwr = alt.Chart(pd.DataFrame({'y': [1.5]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)

# Create the dashed horizontal line at y = 0.8
lower_ewma_acwr = alt.Chart(pd.DataFrame({'y': [0.8]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)
    
chart = alt.Chart(plot_df).mark_bar(color="#0F52BA").encode(
    alt.X("Date:T", axis=alt.Axis(title="Date")),
    alt.Y("Total Distance(m):Q", axis=alt.Axis(title="Total Distance(m)")),
    alt.Tooltip(["Date:T", "Total Distance(m):Q", "Total Distance(m) EWMA ACWR"])
)

ewma_acwr_combined = alt.layer(
    ewma_acwr,
    upper_ewma_acwr,
    lower_ewma_acwr
).resolve_scale(
    y='shared'  # Share y-axis for the EWMA ACWR related charts
)


combined_chart = alt.layer(
    chart,
    ewma_acwr_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title="Total Distance and EWMA ACWR Over Time"
)

In [22]:
combined_chart

alt.LayerChart(...)

In [25]:
def calc_sma_acwr(df, metric, acute_days=7, chronic_days=21, min_periods=21):
    # Function to calculate simple moving average (SMA)
    def sma(series, window):
        return series.rolling(window=window, min_periods=min_periods).mean()
    
    temp_df = df.copy()

    # Calculate SMA ACWR for each player
    temp_df[f'{metric} Acute SMA'] = temp_df.groupby('Player')[metric].transform(lambda x: sma(x, acute_days))
    temp_df[f'{metric} Chronic SMA'] = temp_df.groupby('Player')[metric].transform(lambda x: sma(x, chronic_days))
    df[f'{metric} SMA ACWR'] = (temp_df[f'{metric} Acute SMA'] / temp_df[f'{metric} Chronic SMA']).round(2)

    return df

In [38]:
plot_df = calc_sma_acwr(plot_df, 'Total Distance(m)', acute_days=7, chronic_days=21, min_periods=1)

In [39]:
plot_df.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volume Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Date,Season,21d MA Total Distance(m),Total Distance(m) SMA ACWR
1,Player1,Central Midfielder,Team1,2021,27,2021-W27,51.2,5077.60,480.48,16.0,3.8,14.6,3.2,231.94,41.08,7.527778,15.8,19.0,9.38,99.17,1.28,-0.20,True,Right,54.60%,45.40%,0.75,0.99,0.99,1.37,1.50,1.21,1.14,1.24,1.50,1.02,0.58,0.74,1.19,1.20,1.09,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,1,0,0,2,2021-07-05,2021/22,NaN,1.0
2,Player1,Central Midfielder,Team1,2021,28,2021-W28,55.4,4672.20,501.36,16.0,2.6,16.2,4.8,202.18,12.28,7.611111,24.8,27.6,9.05,84.34,1.53,-0.11,True,Right,52.67%,47.33%,0.91,0.96,1.02,1.13,1.01,1.12,1.22,1.03,0.65,1.01,0.96,1.03,1.06,1.01,1.11,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-12,2021/22,NaN,1.0
3,Player1,Central Midfielder,Team1,2021,29,2021-W29,57.6,4666.34,523.60,18.6,3.6,16.0,6.0,219.10,20.46,7.844444,15.8,16.0,9.09,81.01,1.15,-0.01,False,Right,50.31%,49.69%,0.97,0.98,1.03,1.13,1.15,1.05,1.19,1.05,1.04,1.02,0.77,0.76,1.03,0.98,0.92,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,0,0,0,2,2021-07-19,2021/22,NaN,1.0
4,Player1,Central Midfielder,Team1,2021,30,2021-W30,65.6,5769.14,561.52,16.8,1.4,16.8,7.0,233.08,14.08,7.622222,22.4,25.4,8.56,87.94,1.24,-0.13,True,Right,53.14%,46.86%,1.05,1.09,1.05,1.01,0.63,1.05,1.16,1.05,0.84,1.00,1.04,1.09,0.98,1.03,0.99,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0,2021-07-26,2021/22,NaN,1.0
5,Player1,Central Midfielder,Team1,2021,31,2021-W31,54.5,4107.15,374.20,13.5,2.0,13.0,4.0,81.45,3.00,7.569444,18.0,21.5,6.87,75.36,1.21,-0.19,True,Right,54.43%,45.57%,0.93,0.88,0.82,0.90,0.95,0.90,0.80,0.54,0.31,0.99,0.91,0.96,0.88,0.94,0.98,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-08-02,2021/22,NaN,1.0


In [41]:


sma_acwr = alt.Chart(plot_df).mark_line(point=alt.OverlayMarkDef(color="#40A578"), color="#40A578").encode(
    alt.X("Date:T"),
    alt.Y(f'Total Distance(m) SMA ACWR:Q'),
    alt.Tooltip(["Date:T", "Total Distance(m) SMA ACWR:Q", "Total Distance(m):Q"])
)
# Define the charts
ewma_acwr = alt.Chart(plot_df).mark_line(point=alt.OverlayMarkDef(color="#FF7676"), color="#FA8072").encode(
    alt.X("Date:T"),
    alt.Y(f'Total Distance(m) EWMA ACWR:Q'),
    alt.Tooltip(["Date:T", "Total Distance(m) EWMA ACWR:Q", "Total Distance(m):Q"])
)

# Create the dashed horizontal line at y = 1.5
upper_ewma_acwr = alt.Chart(pd.DataFrame({'y': [1.5]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)

# Create the dashed horizontal line at y = 0.8
lower_ewma_acwr = alt.Chart(pd.DataFrame({'y': [0.8]})).mark_rule(strokeDash=[5, 5], color='#FF6969').encode(
    y='y:Q'
)

# Create the bar chart for Total Distance
chart = alt.Chart(plot_df).mark_bar(color="#0F52BA").encode(
    alt.X("Date:T", axis=alt.Axis(title="Date")),
    alt.Y("Total Distance(m):Q", axis=alt.Axis(title="Total Distance(m)")),
    alt.Tooltip(["Date:T", "Total Distance(m):Q", "Total Distance(m) EWMA ACWR:Q"])
)

ewma_acwr_combined = alt.layer(
    sma_acwr,
    ewma_acwr,
    upper_ewma_acwr,
    lower_ewma_acwr
).resolve_scale(
    y='shared'  # Share y-axis for the EWMA ACWR related charts
)

combined_chart = alt.layer(
    chart,
    ewma_acwr_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title="EWMA/SMA ACWR Over Time",
    width=600,
    height=500
)

combined_chart

alt.LayerChart(...)

In [19]:
metrics_classes

{'Volumn': ['Total Distance(m)', 'Total Player Load', 'Duration'],
 'Intensity': ['Load Per Minute',
  'Distance Per Minute',
  'Acc-Dec-COD Per Minute'],
 'Agility': ['Acc 2m/s2 Total Effort',
  'Acc 3m/s2 Total Effort',
  'Dec 2m/s2 Total Effort',
  'Dec 3m/s2 Total Effort'],
 'IMA': ['IMA COD(left)', 'IMA COD(right)']}

# balance

In [20]:
player_81_w = df_week_player[df_week_player["Player"] == "Player81"]

In [21]:
player_81_w.head()

,Player,Position,Team Name,Year,Week,Year-Week,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score,Date,Season
29468,Player81,Central Midfielder,Team1,2021,26,2021-W26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-06-28,None
29469,Player81,Central Midfielder,Team1,2021,27,2021-W27,50.00,4911.125,530.650,12.00,2.75,15.25,2.00,208.700,60.375,7.097222,16.75,26.75,10.61,98.22,1.42,-0.37,True,Right,61.49%,38.51%,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0,2021-07-05,2021/22
29470,Player81,Central Midfielder,Team1,2021,28,2021-W28,54.75,4778.500,521.875,12.75,1.00,15.00,3.25,177.025,16.625,7.173611,25.25,26.75,9.53,87.28,1.46,-0.06,False,Right,51.44%,48.56%,1.03,0.99,0.99,1.02,0.63,0.99,1.15,0.94,0.53,1.00,1.13,1.00,0.96,0.96,1.01,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0,2021-07-12,2021/22
29471,Player81,Central Midfielder,Team1,2021,29,2021-W29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.03,0.99,0.99,1.02,0.63,0.99,1.15,0.94,0.53,1.00,1.13,1.00,0.96,0.96,1.01,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0,2021-07-19,2021/22
29472,Player81,Central Midfielder,Team1,2021,30,2021-W30,27.50,3547.350,399.800,0.00,0.00,0.00,0.00,0.000,0.000,3.944444,1.50,1.00,14.54,128.99,0.09,0.50,True,Right,40.00%,60.00%,0.80,0.91,0.92,0.00,0.00,0.00,0.00,0.00,0.00,0.82,0.21,0.12,1.10,1.09,0.20,Moderate,Moderate,Moderate,Low,Low,Low,Low,Low,Low,Moderate,Low,Low,Moderate,Moderate,Low,0,1,4,2,2021-07-26,2021/22


In [22]:
player_81_w[["Player","Date","IMA COD(left)", "IMA COD(right)","IMA COD(Right) %", "IMA COD(Left) %", "IMA COD Imbalance"]].head()


,Player,Date,IMA COD(left),IMA COD(right),IMA COD(Right) %,IMA COD(Left) %,IMA COD Imbalance
29468,Player81,2021-06-28,NaN,NaN,nan%,nan%,NaN
29469,Player81,2021-07-05,16.75,26.75,61.49%,38.51%,-0.37
29470,Player81,2021-07-12,25.25,26.75,51.44%,48.56%,-0.06
29471,Player81,2021-07-19,NaN,NaN,nan%,nan%,NaN
29472,Player81,2021-07-26,1.50,1.00,40.00%,60.00%,0.50


In [23]:
player_81_w = player_81_w.dropna()

# Melt the DataFrame to long format for Altair
df_melted = player_81_w.melt(id_vars=['Player', 'Date', 'IMA COD(Right) %', 'IMA COD(Left) %', 'IMA COD Imbalance'], 
                    value_vars=['IMA COD(left)', 'IMA COD(right)'], 
                    var_name='Type', 
                    value_name='Value').dropna()

# Define colors
colors = {'IMA COD(left)': '#0F52BA', 'IMA COD(right)': '#4A8CC7'}

# Create the stack bar chart
bar_chart = alt.Chart(df_melted).mark_bar().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date')),
    y=alt.Y('sum(Value):Q', axis=alt.Axis(title='IMA COD')),
    color=alt.Color('Type:N', scale=alt.Scale(domain=['IMA COD(left)', 'IMA COD(right)'], range=[colors['IMA COD(left)'], colors['IMA COD(right)']])),
    tooltip=[
        alt.Tooltip('Date:T', title='Date'),
        alt.Tooltip('IMA COD(Left) %:N', title='IMA COD(Left) %'),
        alt.Tooltip('IMA COD(Right) %:N', title='IMA COD(Right) %')
    ]
).properties(
    width=600,
    height=400
)

# Create the line chart for IMA COD Imbalance
line_chart = alt.Chart(player_81_w).mark_line(color='#FF7F3E').encode(
    x='Date:T',
    y=alt.Y('IMA COD Imbalance:Q', axis=alt.Axis(title='IMA COD Imbalance', titleColor='#FF7F3E')),
    tooltip=[
        alt.Tooltip('Date:T', title='Date'),
        alt.Tooltip('IMA COD Imbalance:Q', title='IMA COD Imbalance')
    ]
)

# Create the dashed horizontal line at y = 0.8
balance_line = alt.Chart(pd.DataFrame({'y': [0.0]})).mark_rule(strokeDash=[5, 5], color='#FF7F3E').encode(
    y='y:Q'
)

balance_combined = alt.layer(
    line_chart,
    balance_line
).resolve_scale(
    y='shared'  # Share y-axis for IMA COD balance
)


# Combine both charts
combined_chart = alt.layer(
    bar_chart,
    balance_combined
).resolve_scale(
    y='independent'  # Allow independent y-axes
).properties(
    title='IMA COD (Left & Right) and Imbalance'
)

combined_chart

alt.LayerChart(...)

# By Team

In [24]:
# filtered df
filtered_df = df_all[
    (df_all['Team Name'].isin(["Team1"])) &
    (df_all['Date'] == pd.to_datetime("2021-10-20"))
]


In [25]:
filtered_df.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
130051,2021-10-20,Player83,Winger,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.00,0.93,1.01,0.93,0.35,0.59,1.15,0.76,0.84,0.98,0.91,1.08,1.01,0.95,0.92,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
219111,2021-10-20,Player44,Centre Back,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,0.93,0.87,0.88,0.99,0.71,0.96,1.17,1.07,1.52,1.05,0.84,0.88,0.97,0.96,0.97,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,High,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
841,2021-10-20,Player2,Central Midfielder,Team1,65.0,6476.9,834.2,8.0,0.0,13.0,10.0,157.0,4.9,7.638889,26.0,28.0,2021/22,Wednesday,42,2021,2021-W42,12.83,99.64,1.15,-0.07,False,Right,51.85%,48.15%,0.99,0.99,1.00,0.92,0.56,0.97,1.16,1.02,0.81,1.01,0.93,1.00,1.01,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
5221,2021-10-20,Player9,Winger,Team1,73.0,8285.0,979.3,40.0,10.0,32.0,10.0,633.5,60.7,8.333333,29.0,48.0,2021/22,Wednesday,42,2021,2021-W42,13.42,113.49,2.04,-0.40,True,Right,62.34%,37.66%,1.01,1.05,1.04,0.98,0.93,0.99,1.00,1.02,0.91,0.98,0.87,0.95,1.03,1.04,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
132971,2021-10-20,Player39,Goal Keeper,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.13,1.06,1.07,1.22,0.00,1.32,1.60,0.42,NaN,0.99,1.17,1.17,0.94,0.93,1.03,Moderate,Moderate,Moderate,Moderate,Low,Moderate,High,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0


In [26]:
len(filtered_df.dropna())

12

In [27]:
filtered_df["Total Distance(m)"].mean()

6991.174999999999

In [28]:
filtered_df.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
130051,2021-10-20,Player83,Winger,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.00,0.93,1.01,0.93,0.35,0.59,1.15,0.76,0.84,0.98,0.91,1.08,1.01,0.95,0.92,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
219111,2021-10-20,Player44,Centre Back,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,0.93,0.87,0.88,0.99,0.71,0.96,1.17,1.07,1.52,1.05,0.84,0.88,0.97,0.96,0.97,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,High,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
841,2021-10-20,Player2,Central Midfielder,Team1,65.0,6476.9,834.2,8.0,0.0,13.0,10.0,157.0,4.9,7.638889,26.0,28.0,2021/22,Wednesday,42,2021,2021-W42,12.83,99.64,1.15,-0.07,False,Right,51.85%,48.15%,0.99,0.99,1.00,0.92,0.56,0.97,1.16,1.02,0.81,1.01,0.93,1.00,1.01,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
5221,2021-10-20,Player9,Winger,Team1,73.0,8285.0,979.3,40.0,10.0,32.0,10.0,633.5,60.7,8.333333,29.0,48.0,2021/22,Wednesday,42,2021,2021-W42,13.42,113.49,2.04,-0.40,True,Right,62.34%,37.66%,1.01,1.05,1.04,0.98,0.93,0.99,1.00,1.02,0.91,0.98,0.87,0.95,1.03,1.04,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
132971,2021-10-20,Player39,Goal Keeper,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.13,1.06,1.07,1.22,0.00,1.32,1.60,0.42,NaN,0.99,1.17,1.17,0.94,0.93,1.03,Moderate,Moderate,Moderate,Moderate,Low,Moderate,High,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0


In [29]:
filtered_df[["Player", "Total Distance(m)"]].dropna()

,Player,Total Distance(m)
841,Player2,6476.9
5221,Player9,8285.0
41721,Player67,10351.6
97931,Player4,4121.3
22011,Player37,2468.5
51941,Player81,5550.3
72381,Player94,10118.9
111,Player1,11020.9
9601,Player16,3753.2
42451,Player68,4278.7


In [30]:
year_week = filtered_df["Year-Week"].values[0]

In [31]:
selected_teams = ["Team1"]

In [32]:
filtered_df_week = df_all[
    (df_all['Team Name'].isin(selected_teams)) &
    (df_all["Year-Week"] == year_week)
].dropna()

In [33]:
filtered_df_week.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
28579,2021-10-18,Player47,Attacker,Team1,33.0,2873.1,403.7,0.0,0.0,1.0,0.0,0.0,0.0,5.277778,8.0,5.0,2021/22,Monday,42,2021,2021-W42,12.23,87.06,0.42,0.60,True,Right,38.46%,61.54%,0.73,0.74,0.81,0.39,0.32,0.41,0.42,0.33,0.05,0.85,0.55,0.53,1.13,1.02,0.59,Low,Low,Moderate,Low,Low,Low,Low,Low,Low,Moderate,Low,Low,Moderate,Moderate,Low,2,1,4,2
42449,2021-10-18,Player68,Attacker,Team1,71.0,4702.4,494.7,10.0,0.0,14.0,3.0,23.9,0.0,6.361111,24.0,16.0,2021/22,Monday,42,2021,2021-W42,6.97,66.23,0.90,0.50,True,Right,40.00%,60.00%,1.03,1.03,1.03,0.90,0.60,1.02,1.01,0.89,0.73,0.95,1.00,0.95,1.00,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
37339,2021-10-18,Player61,Centre Back,Team1,71.0,5698.3,633.7,16.0,0.0,22.0,4.0,27.1,0.0,7.194444,32.0,28.0,2021/22,Monday,42,2021,2021-W42,8.93,80.26,1.38,0.14,True,Right,46.67%,53.33%,0.98,0.96,0.97,0.97,0.26,1.03,0.77,0.56,0.15,0.98,0.93,0.95,1.01,1.01,0.98,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Low,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
72379,2021-10-18,Player94,Centre Back,Team1,71.0,5284.2,590.0,15.0,1.0,11.0,2.0,13.4,0.0,6.277778,20.0,25.0,2021/22,Monday,42,2021,2021-W42,8.31,74.43,1.00,-0.20,True,Right,55.56%,44.44%,1.03,1.03,1.03,0.96,0.73,0.88,0.79,0.85,0.89,0.97,0.98,0.98,1.00,1.00,0.92,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
49019,2021-10-18,Player77,Goal Keeper,Team1,101.0,5512.1,608.1,10.0,1.0,5.0,5.0,0.0,0.0,5.416667,51.0,48.0,2021/22,Monday,42,2021,2021-W42,6.02,54.58,1.13,0.06,False,Right,48.48%,51.52%,1.04,1.04,1.09,1.12,1.09,0.92,1.29,0.73,0.00,1.00,0.94,0.91,1.03,0.99,0.90,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0


In [34]:
# daily team graph
temp_df = filtered_df[["Player","Duration"]].dropna()

max_series = filtered_df_week.groupby(["Player"])["Duration"].max()
max_series.name = "Max Duration"
max_series = max_series.reset_index()
max_series.rename(columns={'index': 'Player'}, inplace=True)

merged_df = temp_df.merge(max_series, on='Player', how='left')


In [35]:
merged_df

,Player,Duration,Max Duration
0,Player2,65.0,118.0
1,Player9,73.0,93.0
2,Player67,98.0,98.0
3,Player4,70.0,93.0
4,Player37,42.0,65.0
5,Player81,47.0,81.0
6,Player94,98.0,98.0
7,Player1,98.0,98.0
8,Player16,33.0,100.0
9,Player68,51.0,94.0


In [36]:
metric = 'Total Distance(m)'

In [37]:
temp_df = filtered_df[["Player",f"{metric}"]].dropna()

max_series = filtered_df_week.groupby(["Player"])[f"{metric}"].max()
max_series.name = f"Max {metric}"
max_series = max_series.reset_index()
max_series.rename(columns={'index': 'Player'}, inplace=True)

merged_df = temp_df.merge(max_series, on='Player', how='left')
print(merged_df)

merged_df = merged_df.sort_values(by=metric, ascending=False)

# Calculate the average Duration
average_duration = merged_df[metric].mean()

# Create the bar chart for Duration
bars = alt.Chart(merged_df).mark_bar().encode(
    y=alt.Y('Player:N', sort=merged_df['Player'].tolist(), title='Player'),
    x=alt.X(f'{metric}:Q', title=metric),
    tooltip=['Player', metric, f'Max {metric}']
)
# Create the tick chart for Max Duration
ticks = alt.Chart(merged_df).mark_tick(color='red', thickness=2, size=20).encode(
    y=alt.Y('Player:N', sort=merged_df['Player'].tolist(), title='Player'),
    x=alt.X(f'Max {metric}:Q', title=f'{metric} / Max {metric}')
)

# Create the dashed line for average Duration
average_line = alt.Chart(pd.DataFrame({f'Avg {metric}': [average_duration]})).mark_rule(
    strokeDash=[5, 5], color='blue'
).encode(
    x=f'Avg {metric}:Q'
)

# Combine the charts
combined_chart = (bars + ticks + average_line).properties(
    title=f'{metric} vs Weekly Max {metric}'
)


      Player  Total Distance(m)  Max Total Distance(m)
0    Player2             6476.9                11799.0
1    Player9             8285.0                10239.4
2   Player67            10351.6                10351.6
3    Player4             4121.3                11693.6
4   Player37             2468.5                 5353.5
5   Player81             5550.3                 6763.3
6   Player94            10118.9                10430.5
7    Player1            11020.9                11261.6
8   Player16             3753.2                10132.4
9   Player68             4278.7                 9294.7
10  Player53             9596.0                10758.5
11  Player76             7872.8                 7872.8


In [38]:
combined_chart

alt.LayerChart(...)

In [39]:
filtered_df.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
130051,2021-10-20,Player83,Winger,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.00,0.93,1.01,0.93,0.35,0.59,1.15,0.76,0.84,0.98,0.91,1.08,1.01,0.95,0.92,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
219111,2021-10-20,Player44,Centre Back,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,0.93,0.87,0.88,0.99,0.71,0.96,1.17,1.07,1.52,1.05,0.84,0.88,0.97,0.96,0.97,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,High,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
841,2021-10-20,Player2,Central Midfielder,Team1,65.0,6476.9,834.2,8.0,0.0,13.0,10.0,157.0,4.9,7.638889,26.0,28.0,2021/22,Wednesday,42,2021,2021-W42,12.83,99.64,1.15,-0.07,False,Right,51.85%,48.15%,0.99,0.99,1.00,0.92,0.56,0.97,1.16,1.02,0.81,1.01,0.93,1.00,1.01,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
5221,2021-10-20,Player9,Winger,Team1,73.0,8285.0,979.3,40.0,10.0,32.0,10.0,633.5,60.7,8.333333,29.0,48.0,2021/22,Wednesday,42,2021,2021-W42,13.42,113.49,2.04,-0.40,True,Right,62.34%,37.66%,1.01,1.05,1.04,0.98,0.93,0.99,1.00,1.02,0.91,0.98,0.87,0.95,1.03,1.04,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
132971,2021-10-20,Player39,Goal Keeper,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.13,1.06,1.07,1.22,0.00,1.32,1.60,0.42,NaN,0.99,1.17,1.17,0.94,0.93,1.03,Moderate,Moderate,Moderate,Moderate,Low,Moderate,High,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0


In [40]:
filtered_df.groupby("Position")["Total Distance(m)"].mean()

Position
Attacker               6673.70
Central Midfielder     6792.35
Centre Back           10118.90
Full Back                  NaN
Goal Keeper            7872.80
Winger                 6019.10
Name: Total Distance(m), dtype: float64

In [41]:
filtered_df_week.groupby("Position")["Total Distance(m)"].max()

Position
Attacker              11685.9
Central Midfielder    11799.0
Centre Back           10519.6
Full Back              6434.5
Goal Keeper            7872.8
Winger                10239.4
Name: Total Distance(m), dtype: float64

In [42]:
filtered_df_week.groupby("Weekday")["Total Distance(m)"].max()

Weekday
Friday        5748.9
Monday        7260.3
Saturday     11799.0
Thursday      5999.4
Tuesday       4691.7
Wednesday    11020.9
Name: Total Distance(m), dtype: float64

In [43]:
filtered_df_week.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
28579,2021-10-18,Player47,Attacker,Team1,33.0,2873.1,403.7,0.0,0.0,1.0,0.0,0.0,0.0,5.277778,8.0,5.0,2021/22,Monday,42,2021,2021-W42,12.23,87.06,0.42,0.60,True,Right,38.46%,61.54%,0.73,0.74,0.81,0.39,0.32,0.41,0.42,0.33,0.05,0.85,0.55,0.53,1.13,1.02,0.59,Low,Low,Moderate,Low,Low,Low,Low,Low,Low,Moderate,Low,Low,Moderate,Moderate,Low,2,1,4,2
42449,2021-10-18,Player68,Attacker,Team1,71.0,4702.4,494.7,10.0,0.0,14.0,3.0,23.9,0.0,6.361111,24.0,16.0,2021/22,Monday,42,2021,2021-W42,6.97,66.23,0.90,0.50,True,Right,40.00%,60.00%,1.03,1.03,1.03,0.90,0.60,1.02,1.01,0.89,0.73,0.95,1.00,0.95,1.00,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
37339,2021-10-18,Player61,Centre Back,Team1,71.0,5698.3,633.7,16.0,0.0,22.0,4.0,27.1,0.0,7.194444,32.0,28.0,2021/22,Monday,42,2021,2021-W42,8.93,80.26,1.38,0.14,True,Right,46.67%,53.33%,0.98,0.96,0.97,0.97,0.26,1.03,0.77,0.56,0.15,0.98,0.93,0.95,1.01,1.01,0.98,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Low,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
72379,2021-10-18,Player94,Centre Back,Team1,71.0,5284.2,590.0,15.0,1.0,11.0,2.0,13.4,0.0,6.277778,20.0,25.0,2021/22,Monday,42,2021,2021-W42,8.31,74.43,1.00,-0.20,True,Right,55.56%,44.44%,1.03,1.03,1.03,0.96,0.73,0.88,0.79,0.85,0.89,0.97,0.98,0.98,1.00,1.00,0.92,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
49019,2021-10-18,Player77,Goal Keeper,Team1,101.0,5512.1,608.1,10.0,1.0,5.0,5.0,0.0,0.0,5.416667,51.0,48.0,2021/22,Monday,42,2021,2021-W42,6.02,54.58,1.13,0.06,False,Right,48.48%,51.52%,1.04,1.04,1.09,1.12,1.09,0.92,1.29,0.73,0.00,1.00,0.94,0.91,1.03,0.99,0.90,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0


In [44]:
filtered_df_week[["Player", "Total Distance(m)", "Weekday", "Date", "Position"]].head()

,Player,Total Distance(m),Weekday,Date,Position
28579,Player47,2873.1,Monday,2021-10-18,Attacker
42449,Player68,4702.4,Monday,2021-10-18,Attacker
37339,Player61,5698.3,Monday,2021-10-18,Centre Back
72379,Player94,5284.2,Monday,2021-10-18,Centre Back
49019,Player77,5512.1,Monday,2021-10-18,Goal Keeper


In [45]:
filtered_df_week.groupby("Weekday")["Duration"].max().max()

149.0

In [46]:
filtered_df.head()

,Date,Player,Position,Team Name,Duration,Total Distance(m),Total Player Load,Acc 2m/s2 Total Effort,Acc 3m/s2 Total Effort,Dec 2m/s2 Total Effort,Dec 3m/s2 Total Effort,High Intensity Distance(m),Sprint Distance(m),Maximum Velocity(m/s),IMA COD(left),IMA COD(right),Season,Weekday,Week,Year,Year-Week,Load Per Minute,Distance Per Minute,Acc-Dec-COD Per Minute,IMA COD Imbalance,Is IMA Imbalance,IMA COD Deviation,IMA COD(Right) %,IMA COD(Left) %,Duration EWMA ACWR,Total Distance(m) EWMA ACWR,Total Player Load EWMA ACWR,Acc 2m/s2 Total Effort EWMA ACWR,Acc 3m/s2 Total Effort EWMA ACWR,Dec 2m/s2 Total Effort EWMA ACWR,Dec 3m/s2 Total Effort EWMA ACWR,High Intensity Distance(m) EWMA ACWR,Sprint Distance(m) EWMA ACWR,Maximum Velocity(m/s) EWMA ACWR,IMA COD(left) EWMA ACWR,IMA COD(right) EWMA ACWR,Load Per Minute EWMA ACWR,Distance Per Minute EWMA ACWR,Acc-Dec-COD Per Minute EWMA ACWR,is_Duration_abnormal,is_Total Distance(m)_abnormal,is_Total Player Load_abnormal,is_Acc 2m/s2 Total Effort_abnormal,is_Acc 3m/s2 Total Effort_abnormal,is_Dec 2m/s2 Total Effort_abnormal,is_Dec 3m/s2 Total Effort_abnormal,is_High Intensity Distance(m)_abnormal,is_Sprint Distance(m)_abnormal,is_Maximum Velocity(m/s)_abnormal,is_IMA COD(left)_abnormal,is_IMA COD(right)_abnormal,is_Load Per Minute_abnormal,is_Distance Per Minute_abnormal,is_Acc-Dec-COD Per Minute_abnormal,Volumn Risk Score,Intensity Risk Score,Agility Risk Score,IMA Risk Score
130051,2021-10-20,Player83,Winger,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.00,0.93,1.01,0.93,0.35,0.59,1.15,0.76,0.84,0.98,0.91,1.08,1.01,0.95,0.92,Moderate,Moderate,Moderate,Moderate,Low,Low,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0
219111,2021-10-20,Player44,Centre Back,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,0.93,0.87,0.88,0.99,0.71,0.96,1.17,1.07,1.52,1.05,0.84,0.88,0.97,0.96,0.97,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,High,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
841,2021-10-20,Player2,Central Midfielder,Team1,65.0,6476.9,834.2,8.0,0.0,13.0,10.0,157.0,4.9,7.638889,26.0,28.0,2021/22,Wednesday,42,2021,2021-W42,12.83,99.64,1.15,-0.07,False,Right,51.85%,48.15%,0.99,0.99,1.00,0.92,0.56,0.97,1.16,1.02,0.81,1.01,0.93,1.00,1.01,1.00,0.96,Moderate,Moderate,Moderate,Moderate,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,1,0
5221,2021-10-20,Player9,Winger,Team1,73.0,8285.0,979.3,40.0,10.0,32.0,10.0,633.5,60.7,8.333333,29.0,48.0,2021/22,Wednesday,42,2021,2021-W42,13.42,113.49,2.04,-0.40,True,Right,62.34%,37.66%,1.01,1.05,1.04,0.98,0.93,0.99,1.00,1.02,0.91,0.98,0.87,0.95,1.03,1.04,0.93,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,0,0
132971,2021-10-20,Player39,Goal Keeper,Team1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021/22,Wednesday,42,2021,2021-W42,NaN,NaN,NaN,NaN,False,Balance,nan%,nan%,1.13,1.06,1.07,1.22,0.00,1.32,1.60,0.42,NaN,0.99,1.17,1.17,0.94,0.93,1.03,Moderate,Moderate,Moderate,Moderate,Low,Moderate,High,Low,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,Moderate,0,0,2,0


In [54]:
filtered_df[["Player", "Team Name", "Date", "Maximum Velocity(m/s)"]].dropna().sort_values("Maximum Velocity(m/s)", ascending=False)

,Player,Team Name,Date,Maximum Velocity(m/s)
41721,Player67,Team1,2021-10-20,9.027778
111,Player1,Team1,2021-10-20,8.722222
72381,Player94,Team1,2021-10-20,8.555556
5221,Player9,Team1,2021-10-20,8.333333
97931,Player4,Team1,2021-10-20,8.194444
51941,Player81,Team1,2021-10-20,8.111111
9601,Player16,Team1,2021-10-20,8.111111
42451,Player68,Team1,2021-10-20,8.111111
32961,Player53,Team1,2021-10-20,8.000000
48291,Player76,Team1,2021-10-20,7.666667


In [49]:
filtered_df[["Player", "Team Name", "Date", "Maximum Velocity(m/s)"]].head().dropna()

,Player,Team Name,Date,Maximum Velocity(m/s)
841,Player2,Team1,2021-10-20,7.638889
5221,Player9,Team1,2021-10-20,8.333333


In [55]:
team1 = df_all[df_all["Team Name"] == "Team1"]

In [60]:
last_30_days_array = team1.groupby('Player').apply(lambda x: list(x["Maximum Velocity(m/s)"])).reset_index(name='Last 30 Days')

/var/folders/1_/ygw698t54q1fgjfvb_x00lwc0000gn/T/ipykernel_56569/4209136372.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_30_days_array = team1.groupby('Player').apply(lambda x: list(x["Maximum Velocity(m/s)"])).reset_index(name='Last 30 Days')


In [61]:
last_30_days_array

,Player,Last 30 Days
0,Player1,"[7.138888888888888, nan, nan, nan, nan, 7.3888..."
1,Player10,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,Player103,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,Player111,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,Player12,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
5,Player14,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,Player142,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
7,Player147,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
8,Player15,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
9,Player153,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
